In [1]:
from math import *

import numpy

from plur.types import *
from plur.python.data import *

In [2]:
# pick a sample; naturally, this link to my home directory won't work for you
directory = "/home/pivarski/storage/data/TTJets_allevents/0/"

# "AK4CHS" are jets, "Muon" are muons
arrays = {n: numpy.load(directory + n + ".npy") for n in ("events-Lo", "events-Ld-R_Muon-Lo", "events-Ld-R_Muon-Ld-R_pt", "events-Ld-R_Muon-Ld-R_eta", "events-Ld-R_Muon-Ld-R_phi", "events-Ld-R_AK4CHS-Lo", "events-Ld-R_AK4CHS-Ld-R_pt", "events-Ld-R_AK4CHS-Ld-R_eta", "events-Ld-R_AK4CHS-Ld-R_phi")}

# look at the names of what we picked
sorted(arrays.keys(), reverse=True)

['events-Lo',
 'events-Ld-R_Muon-Lo',
 'events-Ld-R_Muon-Ld-R_pt',
 'events-Ld-R_Muon-Ld-R_phi',
 'events-Ld-R_Muon-Ld-R_eta',
 'events-Ld-R_AK4CHS-Lo',
 'events-Ld-R_AK4CHS-Ld-R_pt',
 'events-Ld-R_AK4CHS-Ld-R_phi',
 'events-Ld-R_AK4CHS-Ld-R_eta']

In [3]:
events = fromarrays("events", arrays)
for event in events[:4]:
    print("event: " + repr(event))
    print("    muons: " + " ".join(repr(muon) for muon in event.Muon))
    print("    jets: " + " ".join(repr(jet) for jet in event.AK4CHS))
    print("")

event: <events at 0x0>
    muons: <Muon at 0x0>
    jets: <AK4CHS at 0x0> <AK4CHS at 0x1> <AK4CHS at 0x2> <AK4CHS at 0x3> <AK4CHS at 0x4> <AK4CHS at 0x5>

event: <events at 0x1>
    muons: 
    jets: <AK4CHS at 0x6> <AK4CHS at 0x7> <AK4CHS at 0x8> <AK4CHS at 0x9> <AK4CHS at 0xa> <AK4CHS at 0xb> <AK4CHS at 0xc>

event: <events at 0x2>
    muons: <Muon at 0x1> <Muon at 0x2> <Muon at 0x3>
    jets: <AK4CHS at 0xd> <AK4CHS at 0xe> <AK4CHS at 0xf> <AK4CHS at 0x10> <AK4CHS at 0x11> <AK4CHS at 0x12> <AK4CHS at 0x13> <AK4CHS at 0x14> <AK4CHS at 0x15> <AK4CHS at 0x16>

event: <events at 0x3>
    muons: <Muon at 0x4> <Muon at 0x5> <Muon at 0x6>
    jets: <AK4CHS at 0x17> <AK4CHS at 0x18> <AK4CHS at 0x19> <AK4CHS at 0x1a> <AK4CHS at 0x1b> <AK4CHS at 0x1c> <AK4CHS at 0x1d>



In [29]:
nummuons = numpy.empty(len(events), dtype=numpy.int64)
numjets = numpy.empty(len(events), dtype=numpy.int64)

muonpt1 = numpy.zeros(len(events), dtype=numpy.float32)
jetpt1 = numpy.zeros(len(events), dtype=numpy.float32)

for eventi, event in enumerate(events):
    nummuons[eventi] = len(event.Muon)
    for muon in event.Muon:
        if muon.pt > muonpt1[eventi]:
            muonpt1[eventi] = muon.pt     # assume muons are unsorted (I don't know for sure)

    numjets[eventi] = len(event.AK4CHS)
    for jet in event.AK4CHS:
        if jet.pt > jetpt1[eventi]:
            jetpt1[eventi] = jet.pt       # assume jets are unsorted (I don't know for sure)

In [18]:
muonorder = muonpt1.argsort()[::-1]   # prefer max to min ordering
jetorder = jetpt1.argsort()[::-1]

print(muonorder[:10])
print(jetorder[:10])

[ 3582  9878 25486 37848 15068  9285 18487  8826 22319  9612]
[12665 19515 10279 21718 34930 33891 20522 39713 28183  5027]


In [19]:
print([muon.pt for muon in events[muonorder[0]].Muon])
print([jet.pt for jet in events[jetorder[0]].AK4CHS])

[27280.783, 20.993748, 6.5631175, 3.9093113, 3.6213717, 3.1027567]
[1408.762, 1274.2667, 656.65344, 88.645607, 57.03783, 30.398226, 28.817825]


In [77]:
muonorder_expanded = numpy.empty(len(arrays["events-Ld-R_Muon-Ld-R_pt"]), dtype=muonorder.dtype)
muonbegin = numpy.empty(len(events), dtype=numpy.int64)
muonend = numpy.empty(len(events), dtype=numpy.int64)

index = 0
for sortedi in muonorder:
    muonbegin[sortedi] = index
    for j in range(len(events[sortedi].Muon)):
        muonorder_expanded[index] = arrays["events-Ld-R_Muon-Lo"][sortedi] + j
        index += 1
    muonend[sortedi] = index

In [ ]:
sortedarrays = {
    "sortedevents-Lo":                arrays["events-Lo"],
    "sortedevents-Ld-R_Muon-Lb":      muonbegin,
    "sortedevents-Ld-R_Muon-Le":      muonend,
    "sortedevents-Ld-R_Muon-Ld-R_pt": arrays["events-Ld-R_Muon-Ld-R_pt"][muonorder_expanded]}

sortedevents = fromarrays("sortedevents", sortedarrays)

In [85]:
print([muon.pt for muon in events[0].Muon])
print([muon.pt for muon in sortedevents[0].Muon])

[28.070749]
[28.070749]


In [86]:
print([muon.pt for muon in events[muonorder[0]].Muon])
print([muon.pt for muon in sortedevents[muonorder[0]].Muon])

[27280.783, 20.993748, 6.5631175, 3.9093113, 3.6213717, 3.1027567]
[27280.783, 20.993748, 6.5631175, 3.9093113, 3.6213717, 3.1027567]
